# Data analysis workbook for the vigenere stuff

In [1]:
import cbor2

In [2]:
import scipy.stats

I've copied a couple of files out of `../vigenere/target/criterion/data/` 

- `32early.cbor`, which is the timing run of `vigenere/benches/key_abc32.rs` using the key `012345a`
- `32late.cbor`,  which is the timing run of `vigenere/benches/key_abc32.rs` using the key `tuvwxyz`

Now I need to figure out how to handle them.

Having run `python -m cbor2.tood -p 32early.cbor` on the command line, I see that what I will need to run a t-test is the list of times (in pico seconds?) in the `avg_values` list.

Ok. Time to read the cbor2 docs

I've never tried to read files from a notebook. I hope that file PATH's make sense

In [3]:
EARLY_PATH = './32early.cbor'
LATE_PATH = './32late.cbor'

with open(EARLY_PATH, 'rb') as fe:
    early_data = cbor2.decoder.load(fe)

Now I need to figure out what I have.

In [3]:
type(early_data)

dict

In [4]:
print(early_data.keys())
print(type(early_data["avg_values"]))

print(early_data["avg_values"][0:4])

dict_keys(['datetime', 'iterations', 'values', 'avg_values', 'estimates', 'throughput', 'changes', 'change_direction', 'history_id', 'history_description'])
<class 'list'>
[51081.25, 50266.65, 50691.666666666664, 50281.7625]


Cool. That is what I need.

In [4]:
early_averages = early_data["avg_values"]
print(len(early_averages))

100


In [5]:
with open(LATE_PATH, 'rb') as fl:
    late_data = cbor2.decoder.load(fl)

late_averages = late_data["avg_values"]
len(late_averages)

100

## Stats

Now we start on statistical analysis

In [8]:
(t, p) = scipy.stats.ttest_ind(early_averages,
                                late_averages,
                                equal_var = True,
                                alternative="less")

t is -0.526. p is 0.3


In [9]:
print(f't is {t:#.3}. p is {p:#.3}')


t is -0.526. p is 0.300


I high _p_ value, which we have, doesn't tell us that these are likely to have the same averages. It tells us that we can't rule out that they have the same averages. I hate _p_ values.